In [2]:
# Loading the workspace and create an experiment
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="quick-starts-ws-136655")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136655
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-136655


In [9]:
# Create Compute cluster, use size of VM "StandarD2_V2" in the configuration

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


Using HyperDrive for Hyprparameters Tuning

In [20]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, randint
import os

# Specify parameter sampler
ps = RandomParameterSampling(parameter_space ={
    '--C': uniform(0,1),
    '--max_iter' :randint(200)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = SKLearn(source_directory= './', compute_target=cpu_cluster, entry_script='train.py')
est = SKLearn(source_directory= './', compute_target=compute_target, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                        primary_metric_name='Accuracy',
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                        policy=policy,
                                        estimator=est,
                                        max_total_runs=16,
                                        max_concurrent_runs=4)

NameError: name 'compute_target' is not defined

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

# Starting the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)

# With Jupyter widget, monitoring HyperDrive runs

#RunDetails(hyperdrive_run).show()

#Run completion
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")


NameError: name 'hyperdrive_config' is not defined

In [8]:
import joblib
# Get your best run and save the model from that run.
from azureml.core.model import Model

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()['Accuracy']
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id:  ', best_run.id)
print('Accuracy: ', best_run_metrics)
print('C: ', parameter_values[1])
print('Max Iter: ',parameter_values[3])

# Path specified
best_run.download_file("/outputs/model.joblib","Hyperdrive.joblib")
 


NameError: name 'hyperdrive_run' is not defined

In [20]:
#Best Model 
best_run.get_file_names()
model = best_run.register_model(model_name='Best_BankMarketingModel', model_path='./')

Using AutoML for Hyperparameter Tuning and Model Training

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

csv_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(csv_path)
#

In [13]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)

# view the Cleaned data
df=pd.concat([x, y], axis = 1)
df.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [21]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target = compute_target,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name="y",
    n_cross_validations=5)

NameError: name 'compute_target' is not defined

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

experiment = Experiment(workspace=ws, name="AutoML-project")
autoML_run = experiment.submit(config=automl_config)


# to show how the run is making progress
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()

In [ ]:
# Display Metrics for best run, get dictionary with  details for best run
print("Best Run metrics : ",best_run.getmetrics() )
print("Best Run Details : ", best_run.get_details())

In [ ]:
#Save the best model 
best_run.register_model(model_name = 'myautoml_model.pkl', model_path = './outputs/')

In [ ]:
#Save the resources - delete compute cluster created
compute_target.delete()